In [2]:
#meta 10-11-2023 CSV to GSheets for RapidText

#env myMBpro
#Google Spreadsheets Python API
# conda install -c conda-forge gspread v5.7.1

#notes
#See G Authentication doc for important info

#history
#10/11/2023 AUTOMATE: CSV to GSheets
#     local: get csv
#     myAcct -> GSheet: update w/ changes

#10/30/2023 AUTOMATE: Sort GSheet by Date, FName
#     local: get csv
#     GSheet: update w/ latest data and sort by Date


#References
#G Authentication
# https://docs.gspread.org/en/latest/oauth2.html#authentication

#Examples of gspread Usage
# https://docs.gspread.org/en/v5.10.0/user-guide.html

#CSV to Google Sheets python
# https://stackoverflow.com/questions/74083802/csv-to-google-sheets-python

#How to find the first empty row of a google spread sheet using python GSPREAD?
# refer to https://stackoverflow.com/questions/40781295/how-to-find-the-first-empty-row-of-a-google-spread-sheet-using-python-gspread


In [3]:
import csv
import gspread

In [4]:
gspread.__version__

'5.7.1'

# Automate: CSV to G Sheets

In [5]:
FILE_IN = 'data/myTest.csv'
SHEET_ID = 'my_gsh_test'
SHEET_NAME = 'Sheet1'

## 1. Get CSV Data

In [6]:
f = open(FILE_IN, "r")
values = [r for r in csv.reader(f)]

values[:2]

[['First Name',
  'Middle Name',
  'Last Name',
  'Pref. Phone',
  'Mobile Phone',
  'Date added',
  'Created date',
  'Updated by'],
 ['Mickey',
  '',
  'Mouse',
  '123-123-1234',
  '',
  '2023-06-04',
  '2023-06-04 20:43:36 -0700',
  '']]

## 2. Work with G Sheet
- enable 2 API services  
- connect to G sheet

In [7]:
#authenticate
gc = gspread.oauth()

#connect
sh = gc.open(SHEET_ID)
worksheet = sh.worksheet(SHEET_NAME)

print(sh, worksheet)
#print(sh.sheet1.get('A1'))

<Spreadsheet 'my_gsh_test' id:1X_rTV2PkpFN-5HPsBbreO6Vyiga3d8Q_jQDM8S0GgqY> <Worksheet 'Sheet1' id:0>


- Update G sheet with data from local .csv  
overwrites the entire sheet

In [8]:
#preview headers
worksheet.row_values(1)

['First Name',
 'Middle Name',
 'Last Name',
 'Pref. Phone',
 'Mobile Phone',
 'Date added',
 'Created date',
 'Updated by']

In [9]:
#update by overwriting entire G sheet
d = worksheet.update(values)

In [10]:
#get range, needed for sorting
print(d)
d['updatedRange']

{'spreadsheetId': '1X_rTV2PkpFN-5HPsBbreO6Vyiga3d8Q_jQDM8S0GgqY', 'updatedRange': 'Sheet1!A1:H5', 'updatedRows': 5, 'updatedColumns': 8, 'updatedCells': 40}


'Sheet1!A1:H5'

In [11]:
d['updatedRange'][-2:]

'H5'

In [12]:
worksheet_range = 'A2:' + d['updatedRange'][-2:]
worksheet_range

'A2:H5'

- Prep G sheet for sorting
1) by `date added` 
2) by `first name`

In [13]:
#determine columns to sort
SORT_BY_DATE = 'Date added'
SORT_BY_FNAME = 'First Name'

cell_date = worksheet.find(SORT_BY_DATE)
print("Found it at R%sC%s" % (cell_date.row, cell_date.col))
col_date = cell_date.col

cell_fname = worksheet.find(SORT_BY_FNAME)
print("Found it at R%sC%s" % (cell_fname.row, cell_fname.col))
col_fname = cell_fname.col

print("Sort columns: ", col_date, col_fname)

Found it at R1C6
Found it at R1C1
Sort columns:  6 1


- Sort G sheet

In [14]:
#sort range
worksheet.sort((col_date, 'des'), (col_fname, 'asc'), range=worksheet_range)

{'spreadsheetId': '1X_rTV2PkpFN-5HPsBbreO6Vyiga3d8Q_jQDM8S0GgqY',
 'replies': [{}]}

In [15]:
mystop

NameError: name 'mystop' is not defined

## Xtra

In [ ]:
#$xtra to test
print(sh.sheet1.get('A1'))

In [ ]:
#$xtra determine last row
#How to find the first empty row of a google spread sheet using python GSPREAD?
# refer to https://stackoverflow.com/questions/40781295/how-to-find-the-first-empty-row-of-a-google-spread-sheet-using-python-gspread

def next_available_row(worksheet):
    str_list = list(filter(None, worksheet.col_values(1)))
    return str(len(str_list)+1)

def next_available_column(worksheet):
    str_list = list(filter(None, worksheet.row_values(1)))
    return str(len(str_list)+1)

next_row = next_available_row(worksheet)
last_row = int(next_row) - 1

next_col = next_available_column(worksheet)
last_col = int(next_col) - 1

last_row, last_col